# Imports

In [4]:
import pandas as pd
import numpy as np
import requests
from PyPDF2 import PdfFileReader, PdfFileMerger, PdfFileWriter
import PyPDF2
from bs4 import BeautifulSoup
import os
import shutil
from urllib.parse import urljoin
from urllib.request import Request, urlopen
from io import StringIO, BytesIO
import cloudinary.uploader
import json
from google.cloud import storage

# Loading current banks list

In [23]:
post_req = {"1":{"url":"https://www.abanca.pt", 
                    "name":"ABANCA Corporación Bancaria, S.A.", 
                    "num_pdfs":"3", 
                    "last_updated":"210401", 
                    "sum_sizes":"40", 
                    "bp_bank_id":"0170"}
             ,
              "2":{"url":"https://www.bankinter.pt", 
                    "name":"Bankinter, S.A.", 
                    "num_pdfs":"3", 
                    "last_updated":"210401", 
                    "sum_sizes":"40", 
                    "bp_bank_id":"0269"}
              ,
              "3":{"url":"https://www.bancobic.ao", 
                    "name":"Banco BIC, SA", 
                    "num_pdfs":"3", 
                    "last_updated":"210401", 
                    "sum_sizes":"40", 
                    "bp_bank_id":"0079"}
                ,
                "4":{"url":"https://www.bancobai.ao", 
                    "name":"BAI - Banco Angolano de Investimentos.", 
                    "num_pdfs":"3", 
                    "last_updated":"210401", 
                    "sum_sizes":"40", 
                    "bp_bank_id":"0008"}
              }
post_req = json.dumps(post_req)
post_req

'{"1": {"url": "https://www.abanca.pt", "name": "ABANCA Corporaci\\u00f3n Bancaria, S.A.", "num_pdfs": "3", "last_updated": "210401", "sum_sizes": "40", "bp_bank_id": "0170"}, "2": {"url": "https://www.bankinter.pt", "name": "Bankinter, S.A.", "num_pdfs": "3", "last_updated": "210401", "sum_sizes": "40", "bp_bank_id": "0269"}, "3": {"url": "https://www.bancobic.ao", "name": "Banco BIC, SA", "num_pdfs": "3", "last_updated": "210401", "sum_sizes": "40", "bp_bank_id": "0079"}, "4": {"url": "https://www.bancobai.ao", "name": "BAI - Banco Angolano de Investimentos.", "num_pdfs": "3", "last_updated": "210401", "sum_sizes": "40", "bp_bank_id": "0008"}}'

In [15]:
url = 'http://127.0.0.1:5000/'
do = 'dopdfs'
retrieve = 'retrievepdfs'

In [17]:
# do pdfs
r = requests.post(url+do, data=post_req)
r

<Response [500]>

In [27]:
# collect them // wait a couple of minutes
banks_r = requests.get(url+retrieve)
banks = banks_r.content

In [369]:
banks_dict = json.loads(banks)

In [ ]:
# other request to get: 
## BP pdf url
## concat cloud pdf file url

# Creating the right format for a request

In [381]:
products_request = {'1': {'products':{'1.1': {'url':'https://clientebancario.bportugal.pt/sites/default/files/precario/0170_/0170_PRE.pdf',
                                              'pages': ['5']
                                             },
                '17.1': {'pages': ['5','6']
                        },
                '17.2':{'pages': ['5']
                        },
                '2.1': {'url':'https://clientebancario.bportugal.pt/sites/default/files/precario/0170_/0170_PRE.pdf',
                        'pages': ['5']
                        },
                '18.1': {'pages': ['5']
                        }
    },
     {'commissions':{'1': 'Emissão de extrato',
                     '2': 'Fotocópias de segundas vias de talões de depósito',
                     '3': 'Manutenção de conta', 
                     '4': 'Levantamento de numerário', 
                     '5': 'Adesão ao serviço de banca à distância', 
                     '6': 'Depósito de moedas metálicas',
                     '7': 'Alteração de titulares'
                    }
     },
'2': {'products':{'1.1': {'url':'https://clientebancario.bportugal.pt/sites/default/files/precario/0170_/0170_PRE.pdf',
                          'pages': ['5']
                        },
                '17.1': {'pages': ['5','6']
                        },
                '17.2':{'pages': ['5']
                        },
                '2.1': {'url':'https://clientebancario.bportugal.pt/sites/default/files/precario/0170_/0170_PRE.pdf',
                        'pages': ['5']
                        },
                '18.1': {'pages': ['5']
                        }
    },
     {'commissions':{'1':'Manutenção de conta',
                     '2':'Levantamento de numerário',
                     '3':'Levantamento usd em contas usd',
                     '4': 'Extracto integrado',
                     '5':'Extracto avulso'
                    }
     }
}

In [2]:
banks_dict = {'1': {'bp_bank_id': '0170',
  'bp_pdf_url': 'https://clientebancario.bportugal.pt/sites/default/files/precario/0170_/0170_PRE.pdf',
  'last_updated': '2021-04-06',
  'list_pdfs': ['https://www.abanca.pt/files/documents/precario-folheto-comissoes-823a7663.pdf',
   'https://www.abanca.pt/files/documents/folheto-taxa-juro-precario-5890906f.pdf',
   'https://www.abanca.pt/files/documents/precariodevaloresmobiliarios-23c714a2.pdf',
   'https://www.abanca.pt/files/documents/glossario-6479e89f.pdf',
   'https://www.abanca.pt/files/docs/leaflet-your-rights-payments-eu.pdf'],
  'name': 'ABANCA Corporación Bancaria, S.A.',
  'num_pdfs': 5,
  'price_page': 'https://www.abanca.pt/pt/precario/',
  'sum_sizes': 320,
  'url': 'https://www.abanca.pt'},
 '2': {'bp_bank_id': '0269',
  'bp_pdf_url': 'https://clientebancario.bportugal.pt/sites/default/files/precario/0269_/0269_PRE.pdf',
  'last_updated': '2021-04-06',
  'list_pdfs': ['https://www.bankinter.pt/documents/20182/115991/Pre%C3%A7%C3%A1rio+Completo+TFSI++20210401.pdf/c489f0cd-3831-4441-812e-f351741d2b22',
   'https://banco.bankinter.pt/particulares/pdfs/precario/folha_rosto.pdf',
   'https://banco.bankinter.pt/particulares/pdfs/precario/informacao_geral.pdf',
   'https://banco.bankinter.pt/particulares/pdfs/precario/informacao_complementar.pdf',
   'https://banco.bankinter.pt/particulares/pdfs/precario/p_fcd_contas_deposito.pdf',
   'https://banco.bankinter.pt/particulares/pdfs/precario/p_fcd_operacoes_credito.pdf',
   'https://banco.bankinter.pt/particulares/pdfs/precario/p_fcd_cartoes_credito_debito.pdf',
   'https://banco.bankinter.pt/particulares/pdfs/precario/p_fcd_cheques.pdf',
   'https://banco.bankinter.pt/particulares/pdfs/precario/p_fcd_transferencias.pdf',
   'https://banco.bankinter.pt/particulares/pdfs/precario/p_fcd_cobrancas.pdf',
   'https://banco.bankinter.pt/particulares/pdfs/precario/p_fcd_prestacao_servicos.pdf',
   'https://banco.bankinter.pt/particulares/pdfs/precario/o_fcd_contas_deposito.pdf',
   'https://banco.bankinter.pt/particulares/pdfs/precario/o_fcd_operacoes_credito.pdf',
   'https://banco.bankinter.pt/particulares/pdfs/precario/o_fcd_cartoes_credito_debito.pdf',
   'https://banco.bankinter.pt/particulares/pdfs/precario/o_fcd_cheques.pdf',
   'https://banco.bankinter.pt/particulares/pdfs/precario/o_fcd_transferencias.pdf',
   'https://banco.bankinter.pt/particulares/pdfs/precario/o_fcd_cobrancas.pdf',
   'https://banco.bankinter.pt/particulares/pdfs/precario/o_fcd_prestacao_servicos.pdf',
   'https://banco.bankinter.pt/particulares/pdfs/precario/o_fcd_operacoes_estrangeiro.pdf',
   'https://banco.bankinter.pt/particulares/pdfs/precario/p_ftj_contas_deposito.pdf',
   'https://banco.bankinter.pt/particulares/pdfs/precario/p_ftj_operacoes_credito.pdf',
   'https://banco.bankinter.pt/particulares/pdfs/precario/o_ftj_contas_deposito.pdf',
   'https://banco.bankinter.pt/particulares/pdfs/precario/o_ftj_operacoes_credito.pdf',
   'https://banco.bankinter.pt/particulares/pdfs/precario/ptfs_a.pdf',
   'https://banco.bankinter.pt/particulares/pdfs/precario/ptfs_b.pdf',
   'https://banco.bankinter.pt/particulares/pdfs/precario/ptfs_c.pdf',
   'https://banco.bankinter.pt/particulares/pdfs/precario/ptfs_d.pdf'],
  'name': 'Bankinter, S.A.',
  'num_pdfs': 27,
  'price_page': 'https://www.bankinter.pt/precario1',
  'sum_sizes': 2160,
  'url': 'https://www.bankinter.pt'},
 '3': {'bp_bank_id': '0079',
  'bp_pdf_url': 'https://clientebancario.bportugal.pt/sites/default/files/precario/0079_/0079_PRE.pdf',
  'last_updated': '2021-04-06',
  'list_pdfs': ['https://www.bancobic.ao/dotAsset/0f07d8a4-4ec3-4111-a453-c4ee14d5c66c.pdf',
   'https://www.bancobic.ao/dotAsset/7e1e6dd2-9d56-43ef-b398-ac3bac3cc70e.pdf'],
  'name': 'Banco BIC, SA',
  'num_pdfs': 2,
  'price_page': 'https://www.bancobic.ao/inicio/precario',
  'sum_sizes': 160,
  'url': 'https://www.bancobic.ao'},
 '4': {'bp_bank_id': '0008',
  'bp_pdf_url': 'https://clientebancario.bportugal.pt/sites/default/files/precario/0008_/0008_PRE.pdf',
  'last_updated': '2021-04-06',
  'list_pdfs': ['https://www.bancobai.ao/media/2988/bai_pre_clientes-particulares_31-03-2021.pdf',
   'https://www.bancobai.ao/media/2989/bai_pre_outros-clientes_31-03-2021.pdf',
   'https://www.bancobai.ao/media/2823/bai_resumo-das-alteracoes-do-precario-bna-8-02-2021-cleaned.pdf',
   'https://www.bancobai.ao/media/2278/termos-e-condições.pdf'],
  'name': 'BAI - Banco Angolano de Investimentos.',
  'num_pdfs': 4,
  'price_page': 'https://www.bancobai.ao/pt/preçário',
  'sum_sizes': 240,
  'url': 'https://www.bancobai.ao'}}

In [86]:
banks_dict

{'1': {'bp_bank_id': '0170',
  'bp_pdf_url': 'https://clientebancario.bportugal.pt/sites/default/files/precario/0170_/0170_PRE.pdf',
  'last_updated': '2021-04-06',
  'list_pdfs': ['https://www.abanca.pt/files/documents/precario-folheto-comissoes-823a7663.pdf',
   'https://www.abanca.pt/files/documents/folheto-taxa-juro-precario-5890906f.pdf',
   'https://www.abanca.pt/files/documents/precariodevaloresmobiliarios-23c714a2.pdf',
   'https://www.abanca.pt/files/documents/glossario-6479e89f.pdf',
   'https://www.abanca.pt/files/docs/leaflet-your-rights-payments-eu.pdf'],
  'name': 'ABANCA Corporación Bancaria, S.A.',
  'num_pdfs': 5,
  'price_page': 'https://www.abanca.pt/pt/precario/',
  'sum_sizes': 320,
  'url': 'https://www.abanca.pt',
  'cloud_url': 'https://storage.googleapis.com/bank_price_pdfs/1_all_products.pdf',
  'cloud_url_size': '2129289'},
 '2': {'bp_bank_id': '0269',
  'bp_pdf_url': 'https://clientebancario.bportugal.pt/sites/default/files/precario/0269_/0269_PRE.pdf',
  'l

# Merging and cloud storing all pdfs

In [ ]:
https://storage.cloud.google.com/bank_price_pdfs/testfile.pdf

In [78]:
def upload_file(source_file_bytes, file_name_uploaded, bucket_name="bank_price_pdfs"):
    """Uploads a bytes pdf file to the bucket."""
    print("upload_file was called")
    # source_file_name = "local/path/to/file"
    cloud = 'https://storage.cloud.google.com/'

    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(file_name_uploaded)
#     blob.reload()
    
#     bucket.blob(target_key).upload_from_string(data)

    blob.upload_from_string(source_file_bytes, content_type='application/pdf', timeout = 300.0)

    print(f"file uploaded as {file_name_uploaded}.")
    return blob.public_url

#urljoin(cloud, f'{bucket_name}/{file_name_uploaded}')

In [55]:
pdf_urls = ['https://www.bancobai.ao/media/2988/bai_pre_clientes-particulares_31-03-2021.pdf', 'https://www.abanca.pt/files/documents/precariodevaloresmobiliarios-23c714a2.pdf']
merger = PdfFileMerger()
for pdf in pdf_urls:
    remote = urlopen(Request(pdf)).read()
    memory = BytesIO(remote)
    pdf_file = PdfFileReader(memory)
    merger.append(pdf_file)

merger

In [56]:
tempe = BytesIO()
merger.write(tempe)
upload_file(tempe.getvalue(), 'testfile_bytes.pdf')

upload_file was called
file uploaded as testfile_bytes.pdf.


'https://storage.googleapis.com/bank_price_pdfs/testfile_bytes.pdf'

In [84]:
def pdf_uploader(bucket_name="bank_price_pdfs"):
    for id, values in banks_dict.items():
        print(f'handling pdfs from {values.get("name")}')
        # defining the URLs in a list
        web_pdfs = values.get("list_pdfs")
        # start the merger for each bank
        merger = PdfFileMerger()
        # trying to remotely parse the pdfs all pdf pages
        for pdf_url in web_pdfs:
            try:
                remote = urlopen(Request(pdf_url)).read()
                memory = BytesIO(remote)
                pdf_file = PdfFileReader(memory)
                # check file encryption - if yes, call decrypter
                if pdf_file.isEncrypted:
                    print(f"file {pdf_url} is encrypted. \nstarting to decrypt and adding to merger.")
                    ## pass in web url string of the pdf
                    pdf_file = file_decrypt(pdf_url)
                    merger.append(pdf_file)
                    print(f'added file to pdf merger: {pdf_url}')
                else:
                    print(f"file {pdf_url} is not encrypted. \nadding file to merger...")
                    merger.append(pdf_file)
                    print(f'added file to pdf merger: {pdf_url}')
                    
            except Exception as e:
                print(f"url not found. Error: {e}, url: {pdf_url}\nfile not added to merger.")
                
        # creating a bytes file to be uploaded
        temp = BytesIO()
        merger.write(temp)
        print(f'wrote merger file to BytesIO for: {values.get("name")}')
        print(f'size of BytesIO: {temp.getbuffer().nbytes}')
        values['cloud_url_size'] = f'{temp.getbuffer().nbytes}'
        
        # using gcs uploader to upload bytes file
        file_name_uploaded = f'{id}_all_products.pdf'
        cloud_url = upload_file(temp.getvalue(), file_name_uploaded=file_name_uploaded, bucket_name=bucket_name)
        temp.close()
        merger.close()
        print(f'uploaded merged file to cloud and cleared all memory: {file_name_uploaded}')
        values['cloud_url'] = cloud_url
        print(f'updated banks dict with cloud link: {cloud_url}')

In [85]:
pdf_uploader()

handling pdfs from ABANCA Corporación Bancaria, S.A.
file https://www.abanca.pt/files/documents/precario-folheto-comissoes-823a7663.pdf is encrypted. 
starting to decrypt and adding to merger.
file_decrypt was called for https://www.abanca.pt/files/documents/precario-folheto-comissoes-823a7663.pdf
File decrypted (qpdf)
added file to pdf merger: https://www.abanca.pt/files/documents/precario-folheto-comissoes-823a7663.pdf
file https://www.abanca.pt/files/documents/folheto-taxa-juro-precario-5890906f.pdf is encrypted. 
starting to decrypt and adding to merger.
file_decrypt was called for https://www.abanca.pt/files/documents/folheto-taxa-juro-precario-5890906f.pdf
File decrypted (qpdf)
added file to pdf merger: https://www.abanca.pt/files/documents/folheto-taxa-juro-precario-5890906f.pdf
file https://www.abanca.pt/files/documents/precariodevaloresmobiliarios-23c714a2.pdf is not encrypted. 
adding file to merger...
added file to pdf merger: https://www.abanca.pt/files/documents/precariode

file https://www.bancobic.ao/dotAsset/0f07d8a4-4ec3-4111-a453-c4ee14d5c66c.pdf is not encrypted. 
adding file to merger...
added file to pdf merger: https://www.bancobic.ao/dotAsset/0f07d8a4-4ec3-4111-a453-c4ee14d5c66c.pdf
file https://www.bancobic.ao/dotAsset/7e1e6dd2-9d56-43ef-b398-ac3bac3cc70e.pdf is not encrypted. 
adding file to merger...
added file to pdf merger: https://www.bancobic.ao/dotAsset/7e1e6dd2-9d56-43ef-b398-ac3bac3cc70e.pdf
wrote merger file to BytesIO for: Banco BIC, SA
size of BytesIO: 2804917
upload_file was called
file uploaded as 3_all_products.pdf.
uploaded merged file to cloud and cleared all memory: 3_all_products.pdf
updated banks dict with cloud link: https://storage.googleapis.com/bank_price_pdfs/3_all_products.pdf
handling pdfs from BAI - Banco Angolano de Investimentos.
file https://www.bancobai.ao/media/2988/bai_pre_clientes-particulares_31-03-2021.pdf is not encrypted. 
adding file to merger...
added file to pdf merger: https://www.bancobai.ao/media/298

# looking for relevant pages on PDFs

In [3]:
def file_decrypt(pdf_url, filename="tempe.pdf"):
    """Accepts pdf urls. Returns PyPDF2 FileReader Objects"""
    print(f'file_decrypt was called for {pdf_url}')
    response = requests.get(pdf_url)
    if response.status_code == 200:
        temp = open(filename, "wb")
        temp.write(response.content)
        ## important: have to close again
        temp.close()
        command="cp "+filename+" temp.pdf; qpdf --password='' --decrypt temp.pdf "+filename
        os.system(command)
        print('File decrypted (qpdf)')
        #re-open the decrypted file
        pdfFile = PdfFileReader(filename)
        # removing temp file
        os.remove(filename)
    else: 
        print(f'{url} could not be reached with file_decrypt. Response: {response}')
        return None
    return pdfFile

In [376]:
## searchlist is a list of two strings, one with an id, the other with the topic

def get_relevant_pg(product_num, searchterms=searchterms, exclude=None):
    for id, values in banks_dict.items():
        print(f'handling pdfs from {values.get("url")}')
        print(f'include: {searchterms}, exclude: {exclude}, product number: {product_num}')
        # defining the URLs in a list
        web_pdfs = []
        ## HANDLE SPECIAL CHARS HERE
        web_pdfs = list(values.get("list_pdfs"))
        web_pdfs.insert(0, values.get("bp_pdf_url"))
        # trying to remotely parse the pdfs all pdf pages
        for pdf_url in web_pdfs:
            is_done = False
            try:
                remote = urlopen(Request(pdf_url)).read()
                memory = BytesIO(remote)
                pdf_file = PdfFileReader(memory)
                # check file encryption - if yes, call decrypter
                if pdf_file.isEncrypted:
                    print(f"file {pdf_url} is encrypted. \nstarting to decrypt")
                    ## pass in the memory object or path string
                    pdf_file = file_decrypt(pdf_url)  
                else:
                    print(f"file {pdf_url} is not encrypted. \ncontinuing...")
                    pdf_file = pdf_file
                # iterate through every page in one file to find index page
                for i, page in enumerate(pdf_file.pages):
                    print(f'looking for provided searchterms in page {i+1} of file {pdf_url}')
                    page_text = page.extractText()
                    # validating only the strings of the searchterms
                    search_val = [x in page_text.lower() for x in searchterms]
#                     print(f'search_val {search_val}')
                    excl_val = [x not in page_text.lower() for x in exclude]
#                     print(f'excl_val {excl_val}')
                    num_val = product_num in page_text
#                     print(f'num_val {num_val}')
#                   # validating if one of the strings AND the id is in the page
                    if any(search_val) and all(excl_val) and num_val:
                        print(f'found terms of {searchterms} on page {i+1} of file {pdf_url}')
                        writer = PdfFileWriter()
                        writer.addPage(page)
                        out_path = f'bank_{id}_product_{product_num.replace(".", "_")}.pdf'
                        outfile = open(out_path, 'wb')
                        writer.write(outfile)
                        outfile.close()
                        print(f'saved product page to {out_path}')
                        # closing memory file
                        memory.close
                        is_done = True
                        break
            except Exception as e:
                print(f"url not found. Error: {e}, url: {pdf_url}")
                
            if is_done: 
                break
        if not is_done:
            print(f'could not find {searchterms} of {product_num} in any of these pages: {web_pdfs}')


# Finding page directly

In [377]:
'FOLHETO DE COMISSÕES E DESPESAS'.lower()

'folheto de comissões e despesas'

In [378]:
products = {'1.1': {'include': ['depósitos à ordem'],
                    'exclude': ['folheto de comissões e despesas', 'informação geral']
                   },
            '17.1': {'include': ['depósitos à ordem'],
                    'exclude': ['folheto de comissões e despesas', 'informação geral']
                    },
            '17.2':{'include': ['depósitos a prazo'],
                   'exclude': ['folheto de comissões e despesas', 'informação geral']
                   },
            '2.1': {'include': ['crédito à habitação e crédito hipotecário para outras finalidades'],
                    'exclude': ['folheto de comissões e despesas', 'informação geral']
                   },
            '18.1': {'include': ['crédito à habitação e crédito hipotecário para outras finalidades'],
                    'exclude': ['folheto de comissões e despesas', 'informação geral']
                   }
           }

In [379]:
for num, product in products.items(): 
    print(f'looking for product {num}: {product}')
    include = product.get('include')
    exclude = product.get('exclude')
    get_relevant_pg(product_num=num, searchterms=include, exclude=exclude)

looking for product 1.1: {'include': ['depósitos à ordem'], 'exclude': ['folheto de comissões e despesas', 'informação geral']}
handling pdfs from https://www.abanca.pt
include: ['depósitos à ordem'], exclude: ['folheto de comissões e despesas', 'informação geral'], product number: 1.1
file https://clientebancario.bportugal.pt/sites/default/files/precario/0170_/0170_PRE.pdf is encrypted. 
starting to decrypt
file_decrypt was called for https://clientebancario.bportugal.pt/sites/default/files/precario/0170_/0170_PRE.pdf
File decrypted (qpdf)
looking for provided searchterms in page 1 of file https://clientebancario.bportugal.pt/sites/default/files/precario/0170_/0170_PRE.pdf
looking for provided searchterms in page 2 of file https://clientebancario.bportugal.pt/sites/default/files/precario/0170_/0170_PRE.pdf
looking for provided searchterms in page 3 of file https://clientebancario.bportugal.pt/sites/default/files/precario/0170_/0170_PRE.pdf
looking for provided searchterms in page 4 of

looking for provided searchterms in page 21 of file https://clientebancario.bportugal.pt/sites/default/files/precario/0170_/0170_PRE.pdf
looking for provided searchterms in page 22 of file https://clientebancario.bportugal.pt/sites/default/files/precario/0170_/0170_PRE.pdf
looking for provided searchterms in page 23 of file https://clientebancario.bportugal.pt/sites/default/files/precario/0170_/0170_PRE.pdf
looking for provided searchterms in page 24 of file https://clientebancario.bportugal.pt/sites/default/files/precario/0170_/0170_PRE.pdf
looking for provided searchterms in page 25 of file https://clientebancario.bportugal.pt/sites/default/files/precario/0170_/0170_PRE.pdf
looking for provided searchterms in page 26 of file https://clientebancario.bportugal.pt/sites/default/files/precario/0170_/0170_PRE.pdf
looking for provided searchterms in page 27 of file https://clientebancario.bportugal.pt/sites/default/files/precario/0170_/0170_PRE.pdf
looking for provided searchterms in page 

looking for provided searchterms in page 23 of file https://www.abanca.pt/files/documents/precario-folheto-comissoes-823a7663.pdf
looking for provided searchterms in page 24 of file https://www.abanca.pt/files/documents/precario-folheto-comissoes-823a7663.pdf
looking for provided searchterms in page 25 of file https://www.abanca.pt/files/documents/precario-folheto-comissoes-823a7663.pdf
looking for provided searchterms in page 26 of file https://www.abanca.pt/files/documents/precario-folheto-comissoes-823a7663.pdf
looking for provided searchterms in page 27 of file https://www.abanca.pt/files/documents/precario-folheto-comissoes-823a7663.pdf
looking for provided searchterms in page 28 of file https://www.abanca.pt/files/documents/precario-folheto-comissoes-823a7663.pdf
looking for provided searchterms in page 29 of file https://www.abanca.pt/files/documents/precario-folheto-comissoes-823a7663.pdf
looking for provided searchterms in page 30 of file https://www.abanca.pt/files/documents/

looking for provided searchterms in page 18 of file https://clientebancario.bportugal.pt/sites/default/files/precario/0269_/0269_PRE.pdf
looking for provided searchterms in page 19 of file https://clientebancario.bportugal.pt/sites/default/files/precario/0269_/0269_PRE.pdf
looking for provided searchterms in page 20 of file https://clientebancario.bportugal.pt/sites/default/files/precario/0269_/0269_PRE.pdf
looking for provided searchterms in page 21 of file https://clientebancario.bportugal.pt/sites/default/files/precario/0269_/0269_PRE.pdf
looking for provided searchterms in page 22 of file https://clientebancario.bportugal.pt/sites/default/files/precario/0269_/0269_PRE.pdf
looking for provided searchterms in page 23 of file https://clientebancario.bportugal.pt/sites/default/files/precario/0269_/0269_PRE.pdf
looking for provided searchterms in page 24 of file https://clientebancario.bportugal.pt/sites/default/files/precario/0269_/0269_PRE.pdf
looking for provided searchterms in page 

looking for provided searchterms in page 78 of file https://clientebancario.bportugal.pt/sites/default/files/precario/0269_/0269_PRE.pdf
looking for provided searchterms in page 79 of file https://clientebancario.bportugal.pt/sites/default/files/precario/0269_/0269_PRE.pdf
looking for provided searchterms in page 80 of file https://clientebancario.bportugal.pt/sites/default/files/precario/0269_/0269_PRE.pdf
looking for provided searchterms in page 81 of file https://clientebancario.bportugal.pt/sites/default/files/precario/0269_/0269_PRE.pdf
looking for provided searchterms in page 82 of file https://clientebancario.bportugal.pt/sites/default/files/precario/0269_/0269_PRE.pdf
looking for provided searchterms in page 83 of file https://clientebancario.bportugal.pt/sites/default/files/precario/0269_/0269_PRE.pdf
looking for provided searchterms in page 84 of file https://clientebancario.bportugal.pt/sites/default/files/precario/0269_/0269_PRE.pdf
looking for provided searchterms in page 

...

looking for provided searchterms in page 2 of file https://banco.bankinter.pt/particulares/pdfs/precario/ptfs_c.pdf
looking for provided searchterms in page 3 of file https://banco.bankinter.pt/particulares/pdfs/precario/ptfs_c.pdf
looking for provided searchterms in page 4 of file https://banco.bankinter.pt/particulares/pdfs/precario/ptfs_c.pdf
looking for provided searchterms in page 5 of file https://banco.bankinter.pt/particulares/pdfs/precario/ptfs_c.pdf
file https://banco.bankinter.pt/particulares/pdfs/precario/ptfs_d.pdf is not encrypted. 
continuing...
looking for provided searchterms in page 1 of file https://banco.bankinter.pt/particulares/pdfs/precario/ptfs_d.pdf
looking for provided searchterms in page 2 of file https://banco.bankinter.pt/particulares/pdfs/precario/ptfs_d.pdf
handling pdfs from https://www.bancobic.ao
include: ['crédito à habitação e crédito hipotecário para outras finalidades'], exclude: ['folheto de comissões e despesas', 'informação geral'], product numb

looking for provided searchterms in page 16 of file https://www.bancobic.ao/dotAsset/7e1e6dd2-9d56-43ef-b398-ac3bac3cc70e.pdf
looking for provided searchterms in page 17 of file https://www.bancobic.ao/dotAsset/7e1e6dd2-9d56-43ef-b398-ac3bac3cc70e.pdf
looking for provided searchterms in page 18 of file https://www.bancobic.ao/dotAsset/7e1e6dd2-9d56-43ef-b398-ac3bac3cc70e.pdf
looking for provided searchterms in page 19 of file https://www.bancobic.ao/dotAsset/7e1e6dd2-9d56-43ef-b398-ac3bac3cc70e.pdf
looking for provided searchterms in page 20 of file https://www.bancobic.ao/dotAsset/7e1e6dd2-9d56-43ef-b398-ac3bac3cc70e.pdf
looking for provided searchterms in page 21 of file https://www.bancobic.ao/dotAsset/7e1e6dd2-9d56-43ef-b398-ac3bac3cc70e.pdf
looking for provided searchterms in page 22 of file https://www.bancobic.ao/dotAsset/7e1e6dd2-9d56-43ef-b398-ac3bac3cc70e.pdf
looking for provided searchterms in page 23 of file https://www.bancobic.ao/dotAsset/7e1e6dd2-9d56-43ef-b398-ac3bac3cc

looking for provided searchterms in page 5 of file https://www.bancobai.ao/media/2988/bai_pre_clientes-particulares_31-03-2021.pdf
looking for provided searchterms in page 6 of file https://www.bancobai.ao/media/2988/bai_pre_clientes-particulares_31-03-2021.pdf
looking for provided searchterms in page 7 of file https://www.bancobai.ao/media/2988/bai_pre_clientes-particulares_31-03-2021.pdf
looking for provided searchterms in page 8 of file https://www.bancobai.ao/media/2988/bai_pre_clientes-particulares_31-03-2021.pdf
looking for provided searchterms in page 9 of file https://www.bancobai.ao/media/2988/bai_pre_clientes-particulares_31-03-2021.pdf
looking for provided searchterms in page 10 of file https://www.bancobai.ao/media/2988/bai_pre_clientes-particulares_31-03-2021.pdf
looking for provided searchterms in page 11 of file https://www.bancobai.ao/media/2988/bai_pre_clientes-particulares_31-03-2021.pdf
looking for provided searchterms in page 12 of file https://www.bancobai.ao/media

looking for provided searchterms in page 68 of file https://www.bancobai.ao/media/2988/bai_pre_clientes-particulares_31-03-2021.pdf
looking for provided searchterms in page 69 of file https://www.bancobai.ao/media/2988/bai_pre_clientes-particulares_31-03-2021.pdf
looking for provided searchterms in page 70 of file https://www.bancobai.ao/media/2988/bai_pre_clientes-particulares_31-03-2021.pdf
looking for provided searchterms in page 71 of file https://www.bancobai.ao/media/2988/bai_pre_clientes-particulares_31-03-2021.pdf
looking for provided searchterms in page 72 of file https://www.bancobai.ao/media/2988/bai_pre_clientes-particulares_31-03-2021.pdf
looking for provided searchterms in page 73 of file https://www.bancobai.ao/media/2988/bai_pre_clientes-particulares_31-03-2021.pdf
looking for provided searchterms in page 74 of file https://www.bancobai.ao/media/2988/bai_pre_clientes-particulares_31-03-2021.pdf
looking for provided searchterms in page 75 of file https://www.bancobai.ao/

looking for provided searchterms in page 56 of file https://www.bancobai.ao/media/2989/bai_pre_outros-clientes_31-03-2021.pdf
looking for provided searchterms in page 57 of file https://www.bancobai.ao/media/2989/bai_pre_outros-clientes_31-03-2021.pdf
looking for provided searchterms in page 58 of file https://www.bancobai.ao/media/2989/bai_pre_outros-clientes_31-03-2021.pdf
looking for provided searchterms in page 59 of file https://www.bancobai.ao/media/2989/bai_pre_outros-clientes_31-03-2021.pdf
looking for provided searchterms in page 60 of file https://www.bancobai.ao/media/2989/bai_pre_outros-clientes_31-03-2021.pdf
looking for provided searchterms in page 61 of file https://www.bancobai.ao/media/2989/bai_pre_outros-clientes_31-03-2021.pdf
looking for provided searchterms in page 62 of file https://www.bancobai.ao/media/2989/bai_pre_outros-clientes_31-03-2021.pdf
looking for provided searchterms in page 63 of file https://www.bancobai.ao/media/2989/bai_pre_outros-clientes_31-03-2

# Finding annotations on index

In [212]:
pdf_file = PdfFileReader('../raw_data/0269_PRE.pdf')

In [214]:
page = pdf_file.getPage(1)

In [230]:
pdf_file.getXmpMetadata().pdf_pdfversion

In [233]:
page.extractText().split("\n")

['INFORMAÇÃO GERAL',
 'Reclamações',
 '',
 'Fundo de Garantia de Depósitos',
 'INFORMAÇÃO COMPLEMENTAR',
 'Datas-Valor',
 '1CONTAS DE DEPÓSITO',
 '9CONTAS DE DEPÓSITO',
 '1.1. Depósitos à ordem',
 '9.1. Depósitos à ordem',
 '1.2. Outras modalidades de depósito',
 '9.2. Outras modalidades de depósito',
 '2OPERAÇÕES DE CRÉDITO',
 '10OPERAÇÕES DE CRÉDITO',
 '10.1. Linhas de crédito e contas correntes',
 '2.2. Crédito pessoal',
 '10.2. Descobertos bancários',
 '2.3. Crédito automóvel',
 '10.3. Outros créditos',
 '2.4. Linhas de Crédito e contas Correntes',
 '',
 '2.5. Descobertos bancários',
 '',
 '2.6. Outros créditos a particulares',
 '3CARTÕES DE CRÉDITO E DE DÉBITO',
 '11CARTÕES DE CRÉDITO E DE DÉBITO',
 '3.1. Cartões de crédito',
 '11.1. Cartões de crédito',
 '3.2. Cartões de débito',
 '11.2. Cartões de débito',
 '3.4. Operações com cartões',
 '11.4. Operações com cartões',
 '3.5. Outros serviços com cartões',
 '4CHEQUES',
 '12CHEQUES',
 '4.1. Requisição e entrega de módulos de cheque

In [215]:
key = '/Annots'
uri = '/URI'
ank = '/A'

In [218]:
pageObject = page.getObject()
if key in pageObject.keys():
    ann = pageObject[key]
    for a in ann:
#         print(a)
        u = a.getObject()
        print(u)
        if uri in u[ank].keys():
            print(u[ank][uri])

{'/A': IndirectObject(1579, 0), '/BS': {'/S': '/S', '/Type': '/Border', '/W': 0}, '/Border': [0, 0, 0], '/H': '/I', '/Rect': [40.44, 757.988, 135.934, 768.698], '/Subtype': '/Link', '/Type': '/Annot'}
{'/A': IndirectObject(1580, 0), '/BS': {'/S': '/S', '/Type': '/Border', '/W': 0}, '/Border': [0, 0, 0], '/H': '/I', '/Rect': [224.76, 758.591, 273.537, 767.885], '/Subtype': '/Link', '/Type': '/Annot'}
{'/A': IndirectObject(1581, 0), '/BS': {'/S': '/S', '/Type': '/Border', '/W': 0}, '/Border': [0, 0, 0], '/H': '/I', '/Rect': [224.76, 743.591, 340.737, 752.885], '/Subtype': '/Link', '/Type': '/Annot'}
{'/A': IndirectObject(1582, 0), '/BS': {'/S': '/S', '/Type': '/Border', '/W': 0}, '/Border': [0, 0, 0], '/H': '/I', '/Rect': [40.44, 712.988, 181.571, 723.698], '/Subtype': '/Link', '/Type': '/Annot'}
{'/A': IndirectObject(1583, 0), '/BS': {'/S': '/S', '/Type': '/Border', '/W': 0}, '/Border': [0, 0, 0], '/H': '/I', '/Rect': [224.76, 713.591, 267.155, 722.885], '/Subtype': '/Link', '/Type': '/

In [221]:
for annot in page['/Annots']:
    print(annot.getObject())
    print('')

{'/A': IndirectObject(1579, 0), '/BS': {'/S': '/S', '/Type': '/Border', '/W': 0}, '/Border': [0, 0, 0], '/H': '/I', '/Rect': [40.44, 757.988, 135.934, 768.698], '/Subtype': '/Link', '/Type': '/Annot'}

{'/A': IndirectObject(1580, 0), '/BS': {'/S': '/S', '/Type': '/Border', '/W': 0}, '/Border': [0, 0, 0], '/H': '/I', '/Rect': [224.76, 758.591, 273.537, 767.885], '/Subtype': '/Link', '/Type': '/Annot'}

{'/A': IndirectObject(1581, 0), '/BS': {'/S': '/S', '/Type': '/Border', '/W': 0}, '/Border': [0, 0, 0], '/H': '/I', '/Rect': [224.76, 743.591, 340.737, 752.885], '/Subtype': '/Link', '/Type': '/Annot'}

{'/A': IndirectObject(1582, 0), '/BS': {'/S': '/S', '/Type': '/Border', '/W': 0}, '/Border': [0, 0, 0], '/H': '/I', '/Rect': [40.44, 712.988, 181.571, 723.698], '/Subtype': '/Link', '/Type': '/Annot'}

{'/A': IndirectObject(1583, 0), '/BS': {'/S': '/S', '/Type': '/Border', '/W': 0}, '/Border': [0, 0, 0], '/H': '/I', '/Rect': [224.76, 713.591, 267.155, 722.885], '/Subtype': '/Link', '/Type'

# PyPDF2

In [ ]:
## fpr remote files
remoteFile = urlopen(Request(path)).read()
memoryFile = BytesIO(remoteFile)
pdfFile = PdfFileReader(memoryFile)
writer = PdfFileWriter()

In [ ]:
url = 'https://banco.bankinter.pt/particulares/pdfs/precario/ptfs_d.pdf'

In [ ]:
url = 'https://banco.bankinter.pt/particulares/pdfs/precario/ptfs_d.pdf'

In [ ]:
path = 'bic_all_prices.pdf'

In [ ]:
pdf = PdfFileReader(path)

In [ ]:
def get_index_page(path, search):    
    p = open(path, "rb")
    pdf = PdfFileReader(p)
    page_nums = []
    for page in pdf.pages:
        if search in page.extractText():
            page_nums.append(pdf.getPageNumber(page))
    return page_nums

# Trying other libraries

## PDFX

In [4]:
# import module
import pdfx
  
# reading pdf file
pdf = pdfx.PDFx("https://banco.bankinter.pt/particulares/pdfs/precario/ptfs_d.pdf")
  
# display
print(pdf)

## PDF Miner

In [5]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import StringIO

def convert_pdf_to_txt(path):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    fp = open(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos=set()

    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)

    text = retstr.getvalue()

    fp.close()
    device.close()
    retstr.close()
    return text

In [6]:
path = 'https://clientebancario.bportugal.pt/sites/default/files/precario/0193_/0193_PRE.pdf'

In [7]:
remoteFile = urlopen(Request(path)).read()
memoryFile = BytesIO(remoteFile)
# pdfFile = PdfFileReader(memoryFile)

convert_pdf_to_txt(r.content)

OSError: [Errno 36] File name too long: b'{\n  "abanca": {\n    "pb_bank_id": "0170", \n    "pb_bank_pdf": "https://clientebancario.bportugal.pt/sites/default/files/precario/0170_/0170_PRE.pdf", \n    "pdfs": [\n      "https://www.abanca.pt/files/documents/precario-folheto-comissoes-823a7663.pdf", \n      "https://www.abanca.pt/files/documents/folheto-taxa-juro-precario-5890906f.pdf", \n      "https://www.abanca.pt/files/documents/precariodevaloresmobiliarios-23c714a2.pdf", \n      "https://www.abanca.pt/files/documents/glossario-6479e89f.pdf", \n      "https://www.abanca.pt/files/docs/leaflet-your-rights-payments-eu.pdf"\n    ], \n    "pricelist_url": "https://www.abanca.pt/pt/precario/", \n    "url": "https://www.abanca.pt"\n  }, \n  "bankinter": {\n    "pb_bank_id": "0269", \n    "pb_bank_pdf": "https://clientebancario.bportugal.pt/sites/default/files/precario/0269_/0269_PRE.pdf", \n    "pdfs": [\n      "https://www.bankinter.pt/documents/20182/115991/Pre%C3%A7%C3%A1rio+Completo+TFS

In [8]:
## fpr remote files
remoteFile = urlopen(Request(path)).read()
memoryFile = BytesIO(remoteFile)
pdfFile = PdfFileReader(memoryFile)
writer = PdfFileWriter()

In [ ]:
url = 'https://banco.bankinter.pt/particulares/pdfs/precario/ptfs_d.pdf'

In [37]:
path = 'bic_all_prices.pdf'

In [48]:
pdf = PdfFileReader(path)

In [50]:
pdf.content

AttributeError: 'PdfFileReader' object has no attribute 'content'

In [43]:
def get_rel_page(path, search):    
    p = open(path, "rb")
    pdf = PdfFileReader(p)
    page_nums = []
    for page in pdf.pages:
        if search in page.extractText():
            page_nums.append(pdf.getPageNumber(page))
    return page_nums


In [45]:
regex = r' '
get_rel_page(path, regex)

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77]